In [ ]:
path = r'C:/Users/ruw/Desktop'
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, rf'{path}\RA\penicilin\Libaries')

import reload_lib as rl

import main as m
# from main import main as m
rl.reload_lib(m)

In [51]:
import pandas as pd

In [3]:
matching_database = "ev391cutoff"

In [ ]:
rl.reload_lib(m)


In [49]:
rl.reload_lib(m)
init = m.main(path, matching_database)
db = init.db

# init.import_databases(reload=True,sensitivty=True)

In [219]:
lci_table_template_path = r"C:\Users\ruw\Desktop\RA\penicillin results\LCI_tables.xlsx"
lci_table_template_df = pd.read_excel(lci_table_template_path)
lci_table_template_df

,Reference Flow,Amount,Unit,Provider
0,NaN,NaN,NaN,NaN
1,Input,NaN,NaN,NaN


In [80]:
lci_table_template_df.index  # current index
idx = range(0, len(lci_table_template_df))  # match the number of rows
# To add more indices, you can reindex the DataFrame with a new index of desired length.
# For example, to add one more index (total 3 rows):

new_idx = range(0, 10)  # or any desired length greater than current
lci_table_template_df = lci_table_template_df.reindex(new_idx)
lci_table_template_df

,Reference Flow,Amount,Unit,Provider
0,NaN,NaN,NaN,NaN
1,Input,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [287]:
def remove_empty_rows(df):
    temp = df.isna()

    bio_idx = []
    for idx, val in temp.iterrows():
        if val["Reference Flow"]:
            bio_idx.append(idx)

    df = df.drop(bio_idx)
    return df

In [306]:
dct = {}
lci_table_template_path = r"C:\Users\ruw\Desktop\RA\penicillin results\LCI_tables.xlsx"
for act in db:
    # print(act)
    lci_table_template_df = pd.read_excel(lci_table_template_path)
    new_idx = range(0, len(act.exchanges())+1)  # or any desired length greater than current
    lci_table_template_df = lci_table_template_df.reindex(new_idx)
    bio_idx_start = 0
    for i, exc in enumerate(act.exchanges()):      
        i += 1
        
        for col in lci_table_template_df.columns:
            lci_table_template_df[col] = lci_table_template_df[col].astype('object')

            search_term = col.lower()
            if "Provider" in col:
                search_term = "name"
            if "Reference Flow" in col:
                search_term = "reference product"
            if i == 1 and "production" in exc["type"]:
                lci_table_template_df.loc[0, col] = exc[search_term]
            if "techno" in exc["type"]:
                
                lci_table_template_df.loc[i, col] = exc[search_term]
                if "Provider" in col: 
                    location = exc["location"]
                    lci_table_template_df.loc[i, col] = fr"{exc[search_term]} | {location}"
                bio_idx_start = i
    try:
        bio_idx_start +=1
        
        lci_table_template_df.at[bio_idx_start,"Amount"]
            
        for col in lci_table_template_df.columns:
            lci_table_template_df[col] = lci_table_template_df[col].astype('object')
            for i, exc in enumerate(act.exchanges()):      
                idx = i +bio_idx_start
                search_term = col.lower()
                if "Provider" in col:
                    continue
                if "Reference Flow" in col:
                    search_term = "name"
                if idx == bio_idx_start and col == lci_table_template_df.columns[0]:
                    lci_table_template_df.loc[idx, col] = "Biosphere flow"
                elif "bio" in exc["type"] and i > 0:
                    lci_table_template_df.loc[idx, col] = exc[search_term]
        
        lci_table_template_df = remove_empty_rows(lci_table_template_df)
    except KeyError:
        pass
    dct[act["name"]] = lci_table_template_df

dct["manufacturing of raw penicillium G"]


,Reference Flow,Amount,Unit,Provider
0,raw penicillium G,1,kilogram,manufacturing of raw penicillium G
1,Input,NaN,NaN,NaN
2,"acetone, liquid",0.22,kilogram,"market for acetone, liquid | RoW"
3,ammonium sulfate,0.32,kilogram,market for ammonium sulfate | RoW
4,butyl acetate,0.18,kilogram,market for butyl acetate | RoW
5,"electricity, low voltage",21.977778,kilowatt hour,"market group for electricity, low voltage | IN"
6,glucose,5.18,kilogram,glucose production | RoW
7,"heat, from steam, in chemical industry",6.7155,megajoule,"market for heat, from steam, in chemical indus..."
8,"oxygen, liquid",4.02,kilogram,"market for oxygen, liquid | RoW"
9,pharmamedia,1.3,kilogram,production of pharmamedia | IN


In [312]:
path = r"C:\Users\ruw\Desktop\RA\penicilin\data\LCI_tables.xlsx"

with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
    for act, df in dct.items():
        df.to_excel(writer, sheet_name=act[:31], index=False)


In [313]:
temp = dct["manufacturing of raw penicillium G"]["Reference Flow"].isna().to_frame()

bio_idx = []
for idx, val in temp.iterrows():
    if val["Reference Flow"]:
        bio_idx.append(idx)

bio_idx.append(idx+1)


In [ ]:
dct["manufacturing of raw penicillium G"].drop(bio_idx)

,Reference Flow,Amount,Unit,Provider
0,raw penicillium G,1,kilogram,manufacturing of raw penicillium G
1,Input,NaN,NaN,NaN
2,"acetone, liquid",0.22,kilogram,"market for acetone, liquid"
3,ammonium sulfate,0.32,kilogram,market for ammonium sulfate
4,butyl acetate,0.18,kilogram,market for butyl acetate
5,"electricity, low voltage",21.97777777777778,kilowatt hour,"market group for electricity, low voltage"
6,glucose,5.18,kilogram,glucose production
7,"heat, from steam, in chemical industry",6.7155,megajoule,"market for heat, from steam, in chemical industry"
8,"oxygen, liquid",4.02,kilogram,"market for oxygen, liquid"
9,pharmamedia,1.3,kilogram,production of pharmamedia
